In [ ]:
import numpy as np
import scipy
import sympy
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Collocation, again

The essence of collocation is picking some set of basis functions, and picking the coefficients so that the ODE is solved exactly at some finite set of points.
We showed before that using linear basis functions and picking the collocation point to be either $\tau = 1/2$ or $\tau = 1$ can give good methods for conservative and dissipative problems respectively.
Here we'll do a collocation method of one higher degree -- piecewise quadratic instead of linear functions.
Before, we decided arbitrarily to try 0, 1/2, and 1 as the sole collocation point in the interval.
**How do we pick the collocation points for higher-degree polynomials?**
To answer this question, we have to take a bit of a detour.

### Quadrature

*Quadrature* is the problem of how to approximate integrals.
A *quadrature formula* is a set of points $\{\tau_0, \ldots, \tau_N\}$ and a set of weights $\{\omega_0, \ldots, \omega_N\}$ such that
$$\int_0^1 f(\tau)d\tau \approx \sum_n\omega_nf(\tau_n).$$
We can assume that the interval of integration is always equal to $[0, 1]$ because a general interval can be remapped to the unit interval.

Suppose that we have already picked the quadrature points $\{\tau_0, \ldots, \tau_N\}$.
How do we pick the weights?
One way to do this is to **make the quadrature formula exact for all polynomials up to degree $N$**.
So taking $f_0(\tau) = 1$, we find that the weights have to sum up to 1:
$$\sum_n\omega_n = \sum_n\omega_nf_0(\tau_n) = \int_0^1f_0(\tau)d\tau = 1.$$
If we then take $f_1(\tau) = \tau$, we get our next condition:
$$\sum_n\omega_n\tau_n = \int_0^1f_1(\tau)d\tau = \int_0^1\tau\, d\tau = 1/2.$$
We can continue in this way, taking $f_n(\tau) = \tau^n$ for $n$ up to $N$ to get a complete system of linear equations for the weights.
The matrix for this equation is
$$L_{mn} = \tau_n^m$$
The rows are powers and the columns are quadrature points.
The entries of the right-hand side of the equation we need to solve is
$$f_m = \int_0^1\tau^m d\tau = 1 / (m + 1).$$
Write two procedures to calculate this matrix and right-hand side below.

In [ ]:
def quadrature_matrix(τs):
    L = np.zeros((len(τs), len(τs)))
    ...
    return L

def quadrature_right_hand_side(τs):
    ...

Write some code to test that this works.
Up to you how you do it.
The only things I insist on are that (1) you try it for more than one value of $N$, and (2) for each $N$ that you choose, try it for a polynomial of degree less than or equal to $N$ and another polynomial of degree greater than $N$.
You can pick a polynomial by hand if you want, or you can generate one with random coefficients.
Pick quadrature points however you feel like and if you have time try more than one.
Show the results however you see fit.

So with $N + 1$ points we can integrate polynomials of degree $N$ exactly.
**Can we do better?**
This might seem like a fool's errand.
To see why it isn't, think of quadrature formulas that use only a single point.
For a totally arbitrary choice of point, we can only expect to integrate constant functions (or degree-0 polynomials) exactly.
But if we take the single quadrature point to be 1/2, we can integrate linear functions exactly with, I repeat, only a single point.
Maybe this reminds you of how the forward and backward methods are 1st-order while the midpoint method is 2nd-order...

Are there 2-point quadrature formulas that can integrate quadratic (or maybe even higher degree) polynomials exactly?
Given the number $N$ of points, is there an optimal choice of points?
Amazingly, the answer is yes, but we have to take another detour.

### Legendre polynomials

The *Legendre polynomials* $P_n$ are a set of polynomials with the property that they are all orthogonal with respect to each other:
$$\int_0^1P_m(\tau)P_n(\tau)d\tau = 0$$
if $m$ and $n$ are distinct.
You can obtain the Legendre polynomials by taking the usual monomial basis $1, \tau, \tau^2, \ldots$ and applying the [Gram-Schmidt process](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process).
There's also a handy closed-form expression for them:
$$P_n(\tau) = \frac{1}{n!}\frac{d^n}{d\tau^n}\tau^n(\tau - 1)^n$$
Write some code below to compute a single Legendre polynomial symbolically using the formula above.
The function stub that I've written below takes in the degree $n$ and the symbol $\tau$ as an argument.
We'll create this symbol using sympy below.

In [ ]:
from math import factorial
def legendre_polynomial(n, τ):
    ...

Make a sympy symbol `τ`, compute the Legendre polynomials up to degree 4, save them in a list called `Ps`, and print them out.

In [ ]:
...

The code below will *lambdify* the sympy expressions so that we can evaluate them efficiently and then plot the results.

In [ ]:
τs = np.linspace(0.0, 1.0, 50)
fig, ax = plt.subplots()
ax.plot(τs, np.ones_like(τs), label="degree 0")
for n, P in enumerate(Ps[1:]):
    ax.plot(τs, sympy.lambdify(τ, P)(τs), label=f"degree {n + 1}")
ax.legend();

An important thing to remember is that **the even-degree Legendre polynomials are all equal to 1 at the endpoints, the odd-degree polynomials are equal to -1 on the left and +1 on the right.**

### Gaussian quadrature

That was all very fascinating but what does it have to do with quadrature?
The answer is that **using the roots of the Legendre polynomials, you can integrate polynomials of even higher degree!**
Using any $N + 1$ points, we can always integrate polynomials of degree $N$; but using the roots of $P_{N + 1}$, we can integrate polynomials of degree $2N + 1$.

We can use the sympy.solve function to compute the roots of a polynomial up to degree 4 analytically.
It won't work for degree 5 or higher because [reasons](https://en.wikipedia.org/wiki/Galois_theory).

In [ ]:
P_2 = legendre_polynomial(2, τ)
sympy.solve(P_2, τ)

For higher degrees, we can instead use sympy.nroots to compute the roots numerically.

In [ ]:
P_10 = legendre_polynomial(10, τ)
sympy.nroots(P_10)

Write some code below to return the quadrature points and weights for a Gaussian quadrature formula of degree $N$.
Compute the roots using solve if the degree is 4 or less or numerically if the degree is higher.
Both sympy.solve and sympy.nroots return special sympy data types -- a symbol in the case of .solve, and a speical arbitrary-precision float from nroots.
You might want to take the results, cast them all to `float`, put them in a list, and convert that to a numpy array.
Return the result as a tuple `points, weights`.
The code below assumes that you return the results in this order.

In [ ]:
def gaussian_quadrature_formula(degree):
    ...

The code below will generate the Gaussian quadrature formula and try it on a couple of random polynomials.

In [ ]:
points, weights = gaussian_quadrature_formula(2)
print("∫ f(t) dt ~= " + " + ".join([f"{ω:.3f} * f({τ:.3f})" for τ, ω in zip(points, weights)]))

points, weights = gaussian_quadrature_formula(4)
print("∫ f(t) dt ~= " + " + ".join([f"{ω:.3f} * f({τ:.3f})" for τ, ω in zip(points, weights)]))

In [ ]:
degree = 4

num_trials = 20
errors = np.zeros(num_trials)
for trial in range(num_trials):
    points, weights = gaussian_quadrature_formula(4)
    P = sum(a * τ ** n for n, a in enumerate(rng.normal(size=2 * degree)))
    exact_value = float(sympy.integrate(P, (τ, 0, 1)))
    p = sympy.lambdify(τ, P)
    numeric_value = sum(ω_k * p(τ_k) for τ_k, ω_k in zip(points, weights))
    error = abs(exact_value - numeric_value) / abs(exact_value)
    errors[trial] = error

In [ ]:
print(f"Range of relative errors: {errors.min():.3g}, {errors.max():.3g}")

### Collocation

We've seen quadrature, Legendre polynomials, Gaussian quadrature... what does all this have to do with collocation?
The answer is: **the Gaussian quadrature points make good collocation points too**.
To be specific, we define the **Gauss** schemes of order $n$ for solving ODEs as a collocation method based on degree-n polynomials.
They use the roots of the $n$th Legendre polynomial $P_n$ as collocation points.
Note that all the roots of $P_n$ are contained inside the interval $[0, 1]$.
The **Radau** scheme of order $n$ uses instead the roots of the polynomial $P_n - P_{n - 1}$ as collocation points.
Since all the Legendre polynomials are equal to 1 at $\tau = 1$, this means that the right endpoint of the interval is always a collocation point for the Radau schemes.

The midpoint method, which you will recall works well for wave-type problems, is the order-1 Gauss method.
The backward method, which works well for dissipative or diffusive problems, is the order-1 Radau method.

We'll focus here on how collocation works for the simplest problem there is,
$$\dot z = -\alpha z, \quad z(0) = z_0.$$
This is enough because of the eigendecomposition.
Remember that $\alpha$ might be complex but in order for the equation to be stable we require that the real part of $\alpha$ is greater than or equal to 0.

The purpose of the exercises below is to show you what the Gauss and Radau collocation methods do over a single step.
We're going to write these methods assuming a general choice of polynomial basis and collocation points.
At the end, we'll pick bases and use the collocation points that you computed above.

In the first collocation notebook, we showed that the value of the solution at the next timestep could be expressed in terms of the value at the current timestep:
$$z(t_n + \delta t) = R(\alpha\cdot\delta t)z(t_n).$$
We call $R$ the *stability* function of the method.
In general, it's not a polynomial but the ratio of two polynomials -- a *rational* function.
Note that it only depends on the product $\alpha\cdot\delta t$ of the decay constant and the timestep, not on either quantity individually.

| Method | Stability fn $R(\lambda)$
| ------ | ------------
| forward | $1 - \lambda$
| backward | $\frac{1}{1 + \lambda}$
| midpoint | $\frac{1 - \lambda / 2}{1 + \lambda / 2}$

We call it the stability function because the values of $\lambda$ for which $|R(\lambda)| \le 1$ tells us what timesteps will give us numerical solutions that don't grow for long integration times.
Our goal in the following is to compute the stability function when we use higher-order polynomials.
Here we'll only work with the degree-2 schemes.
We could go higher but this is enough to make our point.

Using the function sympy.solve, compute the collocation points for the Gauss(2) and Radau(2) schemes and save them in variables `gauss_points` and `radau_points`.

In [ ]:
gauss_points = ...
gauss_points

In [ ]:
radau_points = ...
radau_points

### Stability function

If we use a basis $\{p_0, \ldots, p_m\}$, we'll write the expansion for $z$ within the interval as
$$z(t_n + \tau\,\delta t_n) = \sum_ks_kp_k(\tau).$$
We can compute what the stability function is for a collocation method by systematically applying two conditions:
1. The solution $z$ has to be continuous, i.e. the value at the left endpoint of the current interval has to agree with the right endpoint of the previous interval.
2. The ODE has to be exact at the collocation points $\{\tau_1, \ldots, \tau_m\}$.

This will give us a linear system to solve for the coefficients $\{s_k\}$.
The size of the system is only as big as the number of basis functions -- so it's a 3x3 matrix equation if we're using quadratic polynomials.
We will, once again, make sympy do the annoying parts for us.

The first condition is that we should match the old value at the left endpoint:
$$\sum_ks_kp_k(0) = z(t_n),$$
where we assume that $z(t_n)$ is given.
Next, we need that the ODE is exact at the collocation points $\{\tau_1, \ldots, \tau_m\}$.
(Note that we started indexing the basis functions at 0, so there are $m + 1$ basis functions, but there are only $m$ collocation points.)
We can write this as:
$$\sum_ks_k\dot p_k(\tau_j) = -\alpha\,\delta t\,\sum_ks_kp_k(\tau_j).$$
Let's write this as a big linear system:
$$\left\{\left[\begin{matrix}p_0(0) & p_1(0) & \cdots & p_m(0) \\ \dot p_0(\tau_1) & \dot p_1(\tau_1) & \cdots & \dot p_m(0) \\ \vdots & \vdots & & \vdots \\ \dot p_0(\tau_m) & \dot p_1(\tau_m) & \cdots & \dot p_m(\tau_m)\end{matrix}\right] + \alpha\,\delta t\left[\begin{matrix}0 & 0 & \cdots & 0 \\ p_0(\tau_1) & p_1(\tau_1) & \cdots & p_m(\tau_m) \\ \vdots & \vdots & & \vdots \\ p_0(\tau_m) & p_1(\tau_m) & \cdots & p_m(\tau_m)\end{matrix}\right]\right\}\left[\begin{matrix} s_0 \\ s_1 \\ \vdots \\ s_m\end{matrix}\right] = \left[\begin{matrix} z(t_n) \\ 0 \\ \vdots \\ 0 \end{matrix}\right]$$
We'll define the first matrix on the left-hand side as $D$, and the second matrix as $E$.
We can then write this concisely as
$$(D + \alpha\cdot\delta t\cdot E)s = z(t_n)e_0$$
where $e_0$ is the vector that is equal to 1 in the 0th entry and 0 in all other entries.

Fill in the code below to form the $D$ matrix.

In [ ]:
def differentiation_matrix(τ, ps, τs):
    m = len(τs)
    D = sympy.Matrix(m + 1, m + 1, (m + 1) **2 * [0])
    ...
    return D

Fill in the code below to form the $E$ matrix.
Don't include the factor of $\alpha\cdot\delta t$ -- we'll put that in later.

In [ ]:
def evaluation_matrix(τ, ps, τs):
    m = len(τs)
    E = sympy.Matrix(m + 1, m + 1, (m + 1) ** 2 * [0])
    ...
    return E

Finally, having computed all the expansion coefficients $s$, we get the value of $z$ at the next timestep as
$$z(t_n + 1\cdot\delta t) = \sum_ks_kp_k(1).$$
We can then define the *weight* vector $\omega$ as
$$\omega = \left[\begin{matrix}p_0(1) \\ p_1(1) \\ \vdots \\ p_m(1)\end{matrix}\right],$$
in which case we can write
$$z(t_n + 1\cdot\delta t) = \omega^*s.$$
Fill in the body of the function below to compute the weights.

In [ ]:
def weights(τ, ps):
    ...

I'm going to make a choice of basis functions for you.
You can check that these polynomials are a spanning set.

In [ ]:
ps = [(1 - τ)**2, 2 * τ * (1 - τ), τ**2]

Compute the differentiation and evaluation matrices and the weight vector for the Radau points and print them all out.

In [ ]:
D = differentiation_matrix(τ, ps, radau_points)
print(D)
E = evaluation_matrix(τ, ps, radau_points)
print(E)
ω = weights(τ, ps)
print(ω)

If we put together the equations from above together, we find that
$$z(t_n + \delta t) = \omega^*(D + \alpha\cdot\delta t\cdot E)^{-1}e_0\cdot z(t_n).$$
This means that the stability function is
$$R(\lambda) = \omega^*(D + \lambda\cdot E)^{-1}e_0.$$
Fill in the body of the function below to compute the stability function.
Note that this function takes in the symbol $\lambda$ as an additional argument.

In [ ]:
def stability_function(λ, τ, ps, τs):
    ...

The code below computes the stability functions of the Gauss and Radau methods and prints them out.

In [ ]:
λ = sympy.symbols("λ")
R_radau = stability_function(λ, τ, ps, radau_points)
R_radau

In [ ]:
R_gauss = stability_function(λ, τ, ps, gauss_points)
R_gauss

The code below will plot the stability functions for the Gauss and Radau methods for $\lambda$ evenly spaced in the interval [0, 50].
Plot $\exp(-\lambda)$ as well.
What's the asymptotic behavior of each stability function near $\lambda = 0$?
When $\lambda \to \infty$?

In [ ]:
r_radau = sympy.lambdify(λ, R_radau, "numpy")
r_gauss = sympy.lambdify(λ, R_gauss, "numpy")

In [ ]:
λs = np.linspace(0, 50, 250)
fig, ax = plt.subplots()
ax.plot(λs, r_radau(λs), label="Radau")
ax.plot(λs, r_gauss(λs), label="Gauss")
ax.plot(λs, np.exp(-λs), label="$\exp(-\lambda)$")
ax.legend();

Now we take $\lambda$ to be evenly spaced in the interval $i[-10\pi, 10\pi]$, i.e. take $\lambda$ along the imaginary axis and repeat the same exercise.
The plots are of the real and imaginary parts first and include $\exp(-\lambda)$.

In [ ]:
λs = 1j * np.linspace(-10 * np.pi, +10 * np.pi, 500)
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
axes[0].plot(λs.imag, r_radau(λs).real, label="Radau")
axes[0].plot(λs.imag, r_gauss(λs).real, label="Gauss")
axes[0].plot(λs.imag, np.exp(-λs).real, label="$\exp(-\lambda)$")
axes[0].legend()
axes[1].plot(λs.imag, r_radau(λs).imag, label="Radau")
axes[1].plot(λs.imag, r_gauss(λs).imag, label="Gauss")
axes[1].plot(λs.imag, np.exp(-λs).imag, label="$\exp(-\lambda)$")
axes[1].legend();

The plot below shows the stability functions along the imaginary axis in the complex plane.
What do you notice?

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
zs_radau = r_radau(λs)
zs_gauss = r_gauss(λs)
zs = np.exp(-λs)
ax.plot(zs_radau.real, zs_radau.imag, label="Radau")
ax.plot(zs_gauss.real, zs_gauss.imag, label="Gauss")
ax.plot(zs.real, zs.imag, zorder=1, linewidth=5.0, label="$\exp(-\lambda)$")
ax.legend(loc="lower right");

When we take higher-order basis functions, we get a faster convergence rate.
The Radau(n) scheme converges like $\delta t^{2n - 1}$, while the Gauss(n) scheme converges like $\delta t^{2n}$.